# Agenda

1. Iterator protocol
    - Adding iterations to classes
    - Generator functions
    - Generator comprehensions
2. Decorators
3. Threading and multiprocessing

# `__str__` vs. `__repr__`

Both of these methods should return strings. The question is, when is each one run?

Normally `__str__` is run:
- Whenever we run `str` on something
- If we use `print` on something (because it uses `str` behind the scenes)
- In other words: When we want to turn our object into a string, to display it to end users

Normally, `__repr__` is run:
- If we're in a debugger
- If we're in Jupyter (and not using `print`, but just asking for the value of a variable)
- Inside of other data structures (e.g., if we have a list of `Scoop` objects, Python will use `__repr__` and not `__str__` to show us the list)
- In other words: It's meant for developers, not for end users

But:
- If we only define `__str__`, then it does *not* cover cases for `__repr__`
- If we only define `__repr__`, then it *DOES* cover all cases, including those of `__str__`
- In theory, `__repr__` is supposed to return a string that is a valid Python expression.

In [2]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return f'str for Person {vars(self)}'

    def __repr__(self):
        return f'repr for Person {vars(self)}'

    def __format__(self, code):
        return f'format for Person {vars(self)}'
    
p = Person('Reuven')

print(f'{p}')

repr for Person {'name': 'Reuven'}


# Iterator protocol




In [5]:
for one_character in 'abcd':
    print(one_character)

a
b
c
d


In [6]:
for index, one_character in enumerate('abcd'):
    print(f'{index}: {one_character}')

0: a
1: b
2: c
3: d


# What happens in a `for` loop?

- "Iterable" means: Can be put in a `for` loop
- "Iterator" means: We can ask it for its next object, if it has one
Sometimes, an iterable is an iterator, and sometimes it isn't.

- `for` turns to the object and asks if it's iterable. It does this by running the `iter` function on that object. (You will probably never run `iter` in an actual program.)
    - if `iter` returns an object, that object is the "iterator" on which we're going to work
    - Otherwise, we get a `TypeError` exception, indicating that no, the object is not iterable
- Assuming that we get an iterator object back, we call the `next` function on it.
    - We might get something back. That's the value for the current iteration
    - Or we might get `StopIteration`, an exception that says we're at the end of the iterator.
- We keep repeating the above section, calling `next` until we get a `StopIteration` exception.

In [7]:
s = 'abcd'

i = iter(s)   # we ask: is s iterable? If so, return its iterator and assign to i.

In [8]:
i

In [9]:
i = iter(s)
i

In [10]:
next(i)   # ask i for its next value

'a'

In [11]:
 next(i)

'b'

In [12]:
next(i)

'c'

In [13]:
next(i)

'd'

In [14]:
next(i)

StopIteration: 

In [22]:
# Let's create an iterable class!

class MyData:
    def __init__(self, data):
        print(f'\tNow in MyData.__init__')
        self.data = data
        self.index = 0
        
    def __iter__(self):   # this is called each time we *start* a loop on our object -- it returns the iterator
        print('\tNow in MyData.__iter__')
        return self       # meaning: who is my iterator? I'm my own iterator!
    
    def __next__(self):   # this is called once for each iteration
        print(f'\tNow in MyData.__next__: {vars(self)}')
        if self.index >= len(self.data):
            print(f'\tRaising StopIteration')
            raise StopIteration   # no message is needed, because the "for" loop won't read it
            
        value = self.data[self.index]   # get the current value
        self.index += 1                 # increment the counter
        print(f'\tReturning {value}')
        return value                    # return the value for this iteration

m = MyData('abcd')  

for one_item in m:
    print(one_item)

	Now in MyData.__init__
	Now in MyData.__iter__
	Now in MyData.__next__: {'data': 'abcd', 'index': 0}
	Returning a
a
	Now in MyData.__next__: {'data': 'abcd', 'index': 1}
	Returning b
b
	Now in MyData.__next__: {'data': 'abcd', 'index': 2}
	Returning c
c
	Now in MyData.__next__: {'data': 'abcd', 'index': 3}
	Returning d
d
	Now in MyData.__next__: {'data': 'abcd', 'index': 4}
	Raising StopIteration


# Exercise: Circle

1. Create a class, `Circle`, that takes two arguments:
    - `data`, which should be a sequence (string, list, tuple)
    - `n`, the number of iterations we want to get
2. Iterating over an instance of `Circle` will give us `n` results before ending.
3. If the data is shorter than `n` items, then it should automatically return to the start and go from there, as many times as needed.

```python
for one_item in Circle('abc', 7):
    print(one_item)   # a b c a b c a
```

In [27]:
class Circle:
    def __init__(self, data, n):
        self.data = data
        self.n = n
        self.index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= self.n:
            print(f'\tIn __next__, {vars(self)}')
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]  
        self.index += 1
        return value
    
c = Circle('abc', 7)

for one_item in c:
    print(one_item, end=' ')

a b c a b c a 	In __next__, {'data': 'abc', 'n': 7, 'index': 7}


In [28]:
c = Circle('abc', 7)

print('*** A ***')
for one_item in c:
    print(one_item, end=' ')
print()    
    
print('*** B ***')
for one_item in c:
    print(one_item, end=' ')   
print()    

*** A ***
a b c a b c a 	In __next__, {'data': 'abc', 'n': 7, 'index': 7}

*** B ***
	In __next__, {'data': 'abc', 'n': 7, 'index': 7}



In [29]:
class Circle:
    def __init__(self, data, n):
        self.data = data
        self.n = n
        self.index = 0
        
    def __iter__(self):
        self.index = 0
        return self
    
    def __next__(self):
        if self.index >= self.n:
            print(f'\tIn __next__, {vars(self)}')
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]  
        self.index += 1
        return value
    
c = Circle('abc', 7)

print('*** A ***')
for one_item in c:
    print(one_item, end=' ')
print()    
    
print('*** B ***')
for one_item in c:
    print(one_item, end=' ')   
print()    

*** A ***
a b c a b c a 	In __next__, {'data': 'abc', 'n': 7, 'index': 7}

*** B ***
a b c a b c a 	In __next__, {'data': 'abc', 'n': 7, 'index': 7}



In [30]:
s = 'abcd'

i1 = iter(s)
i2 = iter(s)

In [31]:
next(i1)

'a'

In [32]:
next(i1)

'b'

In [33]:
next(i1)

'c'

In [34]:
next(i2)

'a'

In [35]:
next(i1)

'd'

In [36]:
next(i2)

'b'

In [37]:
# let's have Circle.__iter__ return a new instance of a specialized iterator class
# the iterator class will need to implement __next__

class CircleIterator:
    def __init__(self, data, n):
        self.data = data
        self.n = n
        self.index = 0

    def __next__(self):
        if self.index >= self.n:
            print(f'\tIn __next__, {vars(self)}')
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]  
        self.index += 1
        return value

class Circle:
    def __init__(self, data, n):
        self.data = data
        self.n = n
        
    def __iter__(self):
        return CircleIterator(self.data, self.n)
    
    
c = Circle('abc', 7)

print('*** A ***')
for one_item in c:
    print(one_item, end=' ')
print()    
    
print('*** B ***')
for one_item in c:
    print(one_item, end=' ')   
print()    

*** A ***
a b c a b c a 	In __next__, {'data': 'abc', 'n': 7, 'index': 7}

*** B ***
a b c a b c a 	In __next__, {'data': 'abc', 'n': 7, 'index': 7}



In [38]:
i1 = iter(c)
i2 = iter(c)

In [39]:
i1 is i2

False

In [40]:
next(i1)


'a'

In [41]:
next(i1)

'b'

In [42]:
next(i1)

'c'

In [43]:
next(i2)

'a'

# Exercise: MyRange

Write a class, `MyRange`, that emulates the built-in `range` class.  It can take one, two, or three (integer) arguments.  Use a separate helper class as your iterator.

```python
for one_item in MyRange(5):
    print(one_item)   # 0 1 2 3 4
    
for one_item in MyRange(5, 10):
    print(one_item)   # 5 6 7 8 9
    
for one_item in Myrange(5, 20, 3):
    print(one_item)   # 5 8 11 14 17
```

In [46]:
class MyRange:
    def __init__(self, first, second=None, step=1):
        if second is None:
            self.current = 0
            self.stop = first
        else:
            self.current = first
            self.stop = second
        self.step = step
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current >= self.stop:
            raise StopIteration
            
        value = self.current
        self.current += self.step
        return value
    
    
for one_item in MyRange(5):
    print(one_item, end=' ')   # 0 1 2 3 4
print()
    
for one_item in MyRange(5, 10):
    print(one_item, end=' ')   # 5 6 7 8 9
print()
    
for one_item in MyRange(5, 20, 3):
    print(one_item, end=' ')   # 5 8 11 14 17
print()    
        
        

0 1 2 3 4 
5 6 7 8 9 
5 8 11 14 17 


In [47]:
class MyRangeIterator:
    def __init__(self, current, stop, step):
        self.current = current
        self.stop = stop
        self.step = step
    
    def __next__(self):
        if self.current >= self.stop:
            raise StopIteration
            
        value = self.current
        self.current += self.step
        return value
    
class MyRange:
    def __init__(self, first, second=None, step=1):
        if second is None:
            self.current = 0
            self.stop = first
        else:
            self.current = first
            self.stop = second
        self.step = step
        
    def __iter__(self):
        return MyRangeIterator(self.current, self.stop, self.step)
    
for one_item in MyRange(5):
    print(one_item, end=' ')   # 0 1 2 3 4
print()
    
for one_item in MyRange(5, 10):
    print(one_item, end=' ')   # 5 6 7 8 9
print()
    
for one_item in MyRange(5, 20, 3):
    print(one_item, end=' ')   # 5 8 11 14 17
print()    
        
        

0 1 2 3 4 
5 6 7 8 9 
5 8 11 14 17 


In [49]:
class MyRangeIterator:
    def __init__(self, range):
        self.range = range
    
    def __next__(self):
        if self.range.current >= self.range.stop:
            raise StopIteration
            
        value = self.range.current
        self.range.current += self.range.step
        return value
    
class MyRange:
    def __init__(self, first, second=None, step=1):
        if second is None:
            self.current = 0
            self.stop = first
        else:
            self.current = first
            self.stop = second
        self.step = step
        
    def __iter__(self):
        return MyRangeIterator(self)
    
for one_item in MyRange(5):
    print(one_item, end=' ')   # 0 1 2 3 4
print()
    
for one_item in MyRange(5, 10):
    print(one_item, end=' ')   # 5 6 7 8 9
print()
    
for one_item in MyRange(5, 20, 3):
    print(one_item, end=' ')   # 5 8 11 14 17
print()    
        
        

0 1 2 3 4 
5 6 7 8 9 
5 8 11 14 17 


In [52]:
for one_item in range(20, 5, -3):
    print(one_item)

20
17
14
11
8


In [54]:
# allow for negative step size

class MyRangeIterator:
    def __init__(self, range):
        self.range = range
    
    def __next__(self):
        if self.range.step >= 0:
            if self.range.current >= self.range.stop:
                raise StopIteration
        else:
            if self.range.current <= self.range.stop:
                raise StopIteration
            
        value = self.range.current
        self.range.current += self.range.step
        return value
    
class MyRange:
    def __init__(self, first, second=None, step=1):
        if second is None:
            self.current = 0
            self.stop = first
        else:
            self.current = first
            self.stop = second
        self.step = step
        
    def __iter__(self):
        return MyRangeIterator(self)
    
for one_item in MyRange(5):
    print(one_item, end=' ')   # 0 1 2 3 4
print()
    
for one_item in MyRange(5, 10):
    print(one_item, end=' ')   # 5 6 7 8 9
print()
    
for one_item in MyRange(5, 20, 3):
    print(one_item, end=' ')   # 5 8 11 14 17
print()    
        
for one_item in MyRange(20, 5, -3):
    print(one_item, end=' ')   # 5 8 11 14 17
print()    
        

0 1 2 3 4 
5 6 7 8 9 
5 8 11 14 17 
20 17 14 11 8 


In [55]:
class myRangeIterator:
    def __init__(self, start, end, step):
        self.start = start
        self.end = end
        self.step = step
        self.index = 0
        
    def __next__(self): 
        if self.index == self.start and self.end is None:
            raise StopIteration
        elif (self.end - self.start) == self.index and self.step is None:
            raise StopIteration
        elif (self.start + self.step * self.index) > self.end:
            raise StopIteration
        if self.end is None:
            value = self.index
        elif self.step is None:
            value = self.start + self.index
        else:
             value = self.start + self.step
        self.index += 1
        return value             
    
class myRange:
    def __init__(self, start, end=None, step=None):
        self.start = start
        self.end = end
        self.step = step
    def __iter__(self):
        return myRangeIterator(self.start, self.end, self.step)


for one_item in myRange(5):
    print(one_item)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [56]:
import sys
sys.version

'3.10.0 (default, Oct 13 2021, 06:45:00) [Clang 13.0.0 (clang-1300.0.29.3)]'

In [67]:
class Bowl:
    MAX_SCOOPS = 3   # class attribute MAX_SCOOPS

    def __init__(self):
        self.scoops = []

    def add_scoops(self, *args):
        for one_scoop in args:
            if len(self.scoops) >= self.MAX_SCOOPS:
                break
            self.scoops.append(one_scoop)

    def flavors(self):
        return [one_scoop.flavor
                for one_scoop in self.scoops]

    def __repr__(self):
        output = f'{type(self).__name__} of: \n'

        return output + '\n'.join([f'\t{index}: {one_scoop}'
                                   for index, one_scoop in enumerate(self.scoops, 1)])

    def __len__(self):
        return len(self.scoops)

    def __getitem__(self, index):
        # print(f'{index=}')
        return self.scoops[index]

    def __add__(self, other):
        new_bowl = Bowl()
        new_bowl.scoops = self.scoops + other.scoops
        return new_bowl

    def __eq__(self, other):
        return Counter(self.scoops) == Counter(other.scoops)


In [68]:
b = Bowl()
iter(b)

TypeError: 'Bowl' object is not iterable

In [69]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor

    def __repr__(self):
        return f'Scoop of {self.flavor}'


In [70]:
s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
s4 = Scoop('flavor 4')
s5 = Scoop('flavor 5')
s6 = Scoop('flavor 6')


In [71]:
b = Bowl()
b.add_scoops(s1, s2, s3)

for one_item in b:
    print(one_item)

Scoop of chocolate
Scoop of vanilla
Scoop of coffee


# Next up

- Generator functions
- Generator expressions/comprehensions
- Decorators

In [75]:
# Return at :25

In [76]:
# the world's dumbest function

def myfunc():
    return 1
    return 2
    return 3

myfunc()

1

In [77]:
import dis
dis.dis(myfunc)

  4           0 LOAD_CONST               1 (1)
              2 RETURN_VALUE


In [78]:
def myfunc():
    yield 1
    yield 2
    yield 3
    
myfunc()

<generator object myfunc at 0x11095d0e0>

In [79]:
# what is a generator object?  It implements the iterator protocol.

g = myfunc()

type(g)

generator

In [80]:
iter(g)

<generator object myfunc at 0x11095d230>

In [81]:
# what is g's iterator?  A generator!  Meaning: A generator is its own iterator (it returns self in __iter__)

iter(g) is g

True

In [82]:
next(g)

1

In [83]:
next(g)

2

In [84]:
next(g)

3

In [85]:
next(g)

StopIteration: 

# Generator functions

A generator function uses `yield`, and not `return` (except at the end).  When we run the generator function, we get a generator object back. That object is iterable (and also an iterator).

Each time we run `next` on a generator object, the function's body runs through the next `yield` statement, and returns whatever `yield` returns.  Each `next` moves the generator forward one `yield`.

When we reach the end of the function, we get `StopIteration`.  You can actually make that happen faster if you use `return` in the generator function's body.

Each time the generator function has `yield`, it returns a value and goes to sleep at that point.  When we run `next` on it, the function's execution continues from precisely where it left off, after the `yield`.

In [87]:
def myfunc():
    total = 0
    for i in range(5):
        total += i
        yield i, total

In [88]:
g = myfunc()  # running a generator function returns a generator object.  You need to use "next" to run the generator body.

In [89]:
type(g)

generator

In [90]:
next(g)

(0, 0)

In [91]:
next(g)

(1, 1)

In [92]:
next(g)

(2, 3)

In [93]:
next(g)

(3, 6)

In [94]:
next(g)

(4, 10)

In [95]:
next(g)

StopIteration: 

In [96]:
def something():
    return 1
    yield 2
    return 3

In [97]:
something()

<generator object something at 0x11095dbd0>

In [98]:
dis.show_code(something)

Name:              something
Filename:          /var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_20835/4041891837.py
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE
Constants:
   0: None
   1: 1
   2: 2
   3: 3


In [99]:
def fib():
    first = 0
    second = 1
    while True:
        yield first
        first, second = second, first+second
        
for one_number in fib():
    print(one_number, end=' ')
    
    if one_number > 10_000_000_000:
        break

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 

In [100]:
def fib():
    first = 0
    second = 1
    while True:
        yield first
        first, second = second, first+second
        
for one_number in fib():
    print(one_number, end=' ')
    
    if one_number > 10_000_000_000:
        break

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 

In [101]:
def fib():
    first = 0
    second = 1
    while True:
        yield first
        first, second = second, first+second
        
g = fib()
        
for one_number in g:
    print(one_number, end=' ')
    
    if one_number > 10_000_000_000:
        break

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 

In [102]:
next(g)

20365011074

In [103]:
next(g)

32951280099

In [104]:
next(g)

53316291173

In [105]:
list('abcd')

['a', 'b', 'c', 'd']

In [106]:
list((10, 20, 30))

[10, 20, 30]

# Exercise: `read_n`

When we iterate over a file object, we get each line of the file, one at a time. But there are files from which we might want to read two lines at a time, or 10, 3.

The `read_n` generator function takes two arguments:
    - `filename`, the name of the file we'll read from
    - `n`, the number of lines we should return with each iteration
    
The idea is that you can iterate over `read_n(filename, 5)` and know that there are five lines in each iteration.  (The five lines should be returned as one string, with newlines ending each of them.

Two more things:
- If the file ends before you complete `n` lines in the current iteration, then return whatever you've got, and then end the iteration.
- It can help to know that `file.readline()` returns a string with the current line, or an empty string if there are no lines left in the file.

In [107]:
for one_line in open('/etc/passwd'):
    print(len(one_line), end=' ')

3 16 3 76 71 18 2 70 18 3 59 50 54 72 62 64 70 61 71 70 70 72 56 66 62 67 52 63 60 69 58 50 50 54 66 67 59 63 64 61 62 61 62 61 55 55 74 53 65 65 55 56 50 56 88 66 61 70 81 65 62 56 75 65 54 64 75 72 85 72 67 53 55 69 77 74 94 85 97 73 84 68 71 70 63 55 82 74 64 66 76 55 78 80 56 63 82 76 63 55 69 61 99 73 55 63 79 100 57 83 62 77 104 55 67 65 89 60 51 53 

In [121]:
def read_n(filename, n):
    f = open(filename)
    
    while True:
        output = ''
        for _ in range(n):
            output  += f.readline()
    
        if output:  # not an empty string?
            yield output
        else:
            break     # exit the while loop -- and thus the function -- thus raising StopIteration

        
for one_chunk in read_n('/etc/passwd', 13):
    print(one_chunk, end='\n**\n\n')

##
# User Database
# 
# Note that this file is consulted directly only when the system is running
# in single-user mode.  At other times this information is provided by
# Open Directory.
#
# See the opendirectoryd(8) man page for additional information about
# Open Directory.
##
nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
root:*:0:0:System Administrator:/var/root:/bin/sh
daemon:*:1:1:System Services:/var/root:/usr/bin/false

**

_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico
_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false
_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false
_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false
_ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/false
_appstore:*:

In [118]:
def read_n(filename, n):
    f = open(filename)
    
    while True:
        output = ''.join([f.readline()
                          for _ in range(n)])

        if output:  # not an empty string?
            yield output
        else:
            break     # exit the while loop -- and thus the function -- thus raising StopIteration

        
for one_chunk in read_n('/etc/passwd', 13):
    print(one_chunk)

##
# User Database
# 
# Note that this file is consulted directly only when the system is running
# in single-user mode.  At other times this information is provided by
# Open Directory.
#
# See the opendirectoryd(8) man page for additional information about
# Open Directory.
##
nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
root:*:0:0:System Administrator:/var/root:/bin/sh
daemon:*:1:1:System Services:/var/root:/usr/bin/false

_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico
_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false
_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false
_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false
_ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/false
_appstore:*:33:3

In [125]:
def string_add():
    s = ''

    for i in range(100_000):
        s += 'a' * i
        
    return s

In [124]:
def list_add():
    output = []

    for i in range(100_000):
        output.append('a' * i)
        
    return ''.join(output)

In [126]:
%timeit string_add()

2.06 s ± 14.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [127]:
%timeit list_add()

4.26 s ± 93.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [132]:
# using a generator function, we can now basically recreate a for loop!

def myfor(data):
    i = iter(data)  # runs iter to get the iterator
    
    while True:     # runs next repeatedly until getting StopIteration
        try:
            yield next(i)  # yields one value at a time
        except StopIteration:
            break

In [133]:
for one_item in myfor('abcd'):
    print(one_item)

a
b
c
d


In [134]:
# list comprehension
[x**2
 for x in range(-5, 5)]

[25, 16, 9, 4, 1, 0, 1, 4, 9, 16]

In [135]:
# set comprehension
{x**2
 for x in range(-5, 5)}

{0, 1, 4, 9, 16, 25}

In [136]:
# dict comprehension
{x : x**2
 for x in range(-5, 5)}

{-5: 25, -4: 16, -3: 9, -2: 4, -1: 1, 0: 0, 1: 1, 2: 4, 3: 9, 4: 16}

In [138]:
# generator expression or generator comprehension
# it looks like a list comprehension... but returns a generator
# only when we ask for "next" on this object do we get the next object returned
(x**2
 for x in range(-5, 5))

<generator object <genexpr> at 0x110792490>

In [139]:
g = (x**2
 for x in range(-5, 5))

In [140]:
next(g)

25

In [141]:
next(g)


16

In [142]:
next(g)

9

In [144]:
def read_n(filename, n):
    f = open(filename)
    
    while True:
        output = ''.join(f.readline()   # you only need 1 set of () when passing a generator expression to a function
                          for _ in range(n))

        if output:  # not an empty string?
            yield output
        else:
            break     # exit the while loop -- and thus the function -- thus raising StopIteration

        
for one_chunk in read_n('/etc/passwd', 13):
    print(one_chunk)

##
# User Database
# 
# Note that this file is consulted directly only when the system is running
# in single-user mode.  At other times this information is provided by
# Open Directory.
#
# See the opendirectoryd(8) man page for additional information about
# Open Directory.
##
nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
root:*:0:0:System Administrator:/var/root:/bin/sh
daemon:*:1:1:System Services:/var/root:/usr/bin/false

_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico
_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false
_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false
_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false
_ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/false
_appstore:*:33:3

In [ ]:
def pl_word(one_word):
    if one_word[0] in 'aeiou':
        return one_word + 'way'    # sta
    else:
        return one_word[1:] + one_word[0] + 'ay'